In [1]:
import tabula
import pandas as pd
import os
def getList(path) -> pd.DataFrame:
    df = tabula.read_pdf(File,lattice=True,pages="1")
    if len(df[0]) > len(df[1]):
        return df[0]
    else:
        return df[1]

In [44]:
File = r'.\2022\2022.05_出勤簿\PLT2\PLT22\池末.pdf'
os.path.isfile(File)
df = getList(File)
drop = [i for i in range(0,8)]
df = df.drop(drop,axis=0)
col = [i for i in range(0,len(df.columns))]
df.columns = col
df

Got stderr: 7 11, 2022 2:44:05 午前 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
8,NaN,01,日,/法休,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,02,月,/法外,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,03,火,/法外,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,04,水,/法外,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,05,木,/法外,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,06,金,◎出勤,◎0800,7:53,17:37,NLCJ関連業務のため,NaN,8:00,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,07,土,◎出勤,◎0800,7:50,17:15,NaN,NaN,8:00,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,08,日,/法休,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,09,月,◎出勤,◎0800,7:53,17:38,NLCJ関連業務のため,NaN,8:00,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,10,火,◎出勤,◎0800,7:53,19:11,NLCJ関連業務のため,NaN,8:00,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
import datetime
import numpy as np
df_r = pd.DataFrame()
df_r["日付"] = df[1]
df_r["曜日"] = df[2]
df_r["出勤形態"] = df[3]
df_r["出勤定時"] = df[4]
df_r["出勤時間"] = df[5]
df_r["退勤時間"] = df[6]
df_r["申請理由"] = df[7]
df_r["就業時間"] = df[9]
df_r["普通残業"] = df[11]
df_r["深夜残業"] = df[12]
df_r["私的居残り"] = df[19]
# df_r["出勤"] = df[26]
# df_r["年休"] = df[27]
# df_r["代休"] = df[28]
# df_r["看護休暇"] = df[29]
# df_r["特別休暇"] = df[29]
# df_r["欠勤"] = df[29]
# df_r["他"] = df[29]
drop = [i for i in range(31+8,44)]
df_r = df_r.drop(drop,axis=0)
df_r["夜中フラグ"] = df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"].str.contains("翌")
df_r["退勤時間"] = df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"].str.replace("翌","")
df_r["退勤時間"] = pd.to_datetime(df_r[~pd.isna(df_r["退勤時間"])]["退勤時間"])
df_r["夜中フラグ"] = df_r["夜中フラグ"].fillna(False)
df_r["出勤フラグ"] = df_r["出勤形態"] == "◎出勤"

In [80]:
df_r["勤務時間"] = df_r["出勤定時"].apply(lambda x: pd.to_datetime('17:00') if x=="◎0800" else None)

In [81]:
df_r["超過勤務"] = df_r["退勤時間"] - df_r["勤務時間"]
df_r["超過勤務"] = df_r["超過勤務"].apply(lambda x:x+datetime.timedelta(days=1) if x < datetime.timedelta(days=0) else x)

In [82]:
df_r["普通残業"] = df_r["普通残業"].apply(lambda x:False if x == np.nan else pd.to_datetime(x))
df_r["深夜残業"] = df_r["深夜残業"].apply(lambda x:False if x == np.nan else pd.to_datetime(x))
df_r["深夜残業"] = df_r["深夜残業"].apply(lambda x:False if x == np.nan else pd.to_datetime(x))

In [83]:
df_r

,日付,曜日,出勤形態,出勤定時,出勤時間,退勤時間,申請理由,就業時間,普通残業,深夜残業,私的居残り,夜中フラグ,出勤フラグ,勤務時間,超過勤務
8,01,日,/法休,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
9,02,月,/法外,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
10,03,火,/法外,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
11,04,水,/法外,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
12,05,木,/法外,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
13,06,金,◎出勤,◎0800,7:53,2022-07-11 17:37:00,NLCJ関連業務のため,8:00,2022-07-11 00:37:00,NaN,NaN,False,True,2022-07-11 17:00:00,0 days 00:37:00
14,07,土,◎出勤,◎0800,7:50,2022-07-11 17:15:00,NaN,8:00,NaT,NaN,0:15,False,True,2022-07-11 17:00:00,0 days 00:15:00
15,08,日,/法休,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,False,False,NaT,NaT
16,09,月,◎出勤,◎0800,7:53,2022-07-11 17:38:00,NLCJ関連業務のため,8:00,2022-07-11 00:38:00,NaN,NaN,False,True,2022-07-11 17:00:00,0 days 00:38:00
17,10,火,◎出勤,◎0800,7:53,2022-07-11 19:11:00,NLCJ関連業務のため,8:00,2022-07-11 02:11:00,NaN,NaN,False,True,2022-07-11 17:00:00,0 days 02:11:00


In [197]:
pd.to_datetime(df_r[~pd.isna(df_r["退勤時間"]) & df_r["退勤時間"].str.contains("翌")]["退勤時間"].str.replace("翌",""))

18   2022-07-07 00:11:00
24   2022-07-07 00:41:00
Name: 退勤時間, dtype: datetime64[ns]

In [ ]:
df_r["退勤時間"].